## How to create a CNN in {tool}?

In [ ]:
import datetime

import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

### Toy Dataset

In [ ]:
train_df = pd.read_csv('fashion-mnist/fashion-mnist_train.csv')
test_df = pd.read_csv('fashion-mnist/fashion-mnist_test.csv')

In [ ]:
x_train = train_df.values[:, 1:] / 255
y_train = train_df.values[:, 0]

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=.8)
x_train = x_train.reshape(-1, 1, 28, 28)
x_valid = x_valid.reshape(-1, 1, 28, 28)

x_test = test_df.values[:, 1:] / 255
x_test = x_test.reshape(-1, 1, 28, 28)
y_test = test_df.values[:, 0]

### PyTorch

In [ ]:
import torch
import pytorch_lightning as pl
import torch.nn.functional as F

from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train).long())
train_dl = DataLoader(train_ds, batch_size=4)

valid_ds = TensorDataset(torch.Tensor(x_valid), torch.Tensor(y_valid).long())
valid_dl = DataLoader(valid_ds, batch_size=4)

In [ ]:
class TorchNN(pl.LightningModule):
    def __init__(self, lr: float = 1e-3):
        super(TorchNN, self).__init__()
        self.lr = lr
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = F.cross_entropy(y_hat, y)
        return val_loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)

In [ ]:
nn_torch = TorchNN()
trainer = pl.Trainer(max_epochs=4)

In [ ]:
trainer.fit(nn_torch, train_dl, valid_dl)

Verificamos o modelo em
`tensorboard --logdir lightning_logs/`  
Referências em [Next Jornal: PyTorch MNIST](https://nextjournal.com/gkoehler/pytorch-mnist)

### TensorFlow

In [ ]:
import tensorflow as tf

In [ ]:
load_ext tensorboard

In [ ]:
log_dir = "tensorflow_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
class FlowNN(tf.keras.Model):
    def __init__(self):
        super(FlowNN, self).__init__()
        inputs = tf.keras.layers.Input(shape=[28, 28, 1])
        #
        conv1 = tf.keras.layers.Conv2D(10, 5, activation='relu')(inputs)
        conv1_pool = tf.keras.layers.MaxPool2D()(conv1)
        #
        conv2 = tf.keras.layers.Conv2D(20, 5)(conv1_pool)
        conv2_drop = tf.keras.layers.Dropout(rate=.5)(conv2)
        conv2_pool = tf.keras.layers.MaxPool2D()(conv2_drop)
        conv2_relu = tf.keras.layers.ReLU()(conv2_pool)
        #
        reshape = tf.keras.layers.Reshape((-1, 320))(conv2_relu)
        #
        fc1 = tf.keras.layers.Dense(50, activation='relu')(reshape)
        fc1_drop = tf.keras.layers.Dropout(rate=.5)(fc1)
        #
        outputs = tf.keras.layers.Dense(10)(fc1_drop)
        #
        self.model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    
    def call(self, inputs):
        return self.model(inputs)

In [ ]:
flow_nn = FlowNN()

In [ ]:
flow_nn.compile(
    loss=tf.nn.sparse_softmax_cross_entropy_with_logits,
    optimizer='Adam',
    metrics=['accuracy']
)

In [ ]:
x_train_flow = np.transpose(x_train, (0, 2, 3, 1))
x_valid_flow = np.transpose(x_valid, (0, 2, 3, 1))

In [ ]:
flow_nn.fit(
    x_train_flow,
    y_train,
    epochs=4,
    validation_data=(x_valid_flow, y_valid),
    callbacks=[tensorboard_callback]
)

Verificamos o modelo em `tensorboard --logdir tensorflow_logs`